# Import Required Libraries
Import the necessary libraries, including requests and pandas.

In [0]:
# Importing the required libraries
import requests
import pandas as pd
from pyinaturalist import get_observations

# Send GET Request to iNaturalist API
Use the requests library to send a GET request to the iNaturalist API.

In [0]:
laval_place_id = 27655
montreal_place_id = 187355
gatineau_place_id = 142292

# Initialize an empty DataFrame to store the observations
df = pd.DataFrame()
# Iterate over each page and append the data to the DataFrame
for page in range(1, 10):
    # Get the observations for the specified place and page
    laval_observations = get_observations(place_id=laval_place_id, verifiable=True, page=str(page), quality_grade="research")
    # Append the observations to the DataFrame
    df = df.append(laval_observations["results"], ignore_index=True)

# Save the DataFrame to a CSV file
df.to_csv('inaturalist_data.csv', index=False)

print("Data saved to 'inaturalist_data.csv'.")

In [0]:
import os

# Get all files in the data folder
# Read the 'sentinelle_liste_sp.csv' file from the workspace
file_path = 'sentinelle_liste_sp.csv'
df_sentinelle = pd.read_csv(file_path)
df_sentinelle["Nom_francais"] = df_sentinelle[['Nom_francais']].apply(lambda x: x.astype(str).str.lower())
# Filter the files based on the ones that contain "sentinelle" in their name

display(df_sentinelle)
# Add any additional code here for further processing or analysis

In [0]:
# Keep only the 'species_guess' column and add a new column with the value 'Laval'
# Convert the values in the 'species_guess' column to lowercase and select only the 'species_guess' column
df_simplified = df[['species_guess']].apply(lambda x: x.astype(str).str.lower())
df_simplified['location'] = 'Laval'

display(df_simplified)

df_simplified.to_csv("laval_with_species.csv")

In [0]:
# Perform a left join on the specified columns
merged_df = df_simplified.merge(right=df_sentinelle, left_on='species_guess', right_on='Nom_francais', how='inner')
# Display the merged DataFrame
display(merged_df)

In [0]:
display(df)

In [0]:
# Specify the column values to filter on
values_to_filter = df["species_guess"]

# Filter the DataFrame based on the values in the specified column
filtered_df = df[df_sentinelle['Nom_francais'].isin(values_to_filter)]

display(filtered_df)

# Parse API Response
Parse the JSON response from the API and convert it into a pandas DataFrame.

In [0]:
# Parse the JSON response and convert it into a pandas DataFrame
# Extract the relevant information from each observation
observations = []
for result in results:
    observation = {
        'id': result['id'],
        'observed_on': result['observed_on'],
        'description': result['description'],
        'latitude': result['geojson']['coordinates'][1],
        'longitude': result['geojson']['coordinates'][0],
        'location': result['location'],
        'species_guess': result['species_guess']
    }
    observations.append(observation)

# Convert the list of observations into a pandas DataFrame
df = pd.DataFrame(observations)

# Save the DataFrame to a CSV file
df.to_csv('inaturalist_data.csv', index=False)

print("Data saved to 'inaturalist_data.csv'.")

# Filter Data by City
Filter the DataFrame based on the city or cities of interest.

In [0]:
# Define the cities of interest
cities_of_interest = ['city1', 'city2', 'city3']  # replace with your cities

# Filter the DataFrame based on the city or cities of interest
df_filtered = df[df['location'].isin(cities_of_interest)]

# Save the filtered DataFrame to a CSV file
df_filtered.to_csv('filtered_inaturalist_data.csv', index=False)

print("Filtered data saved to 'filtered_inaturalist_data.csv'.")

# Save Data to CSV
Save the filtered data to a CSV file using pandas.

In [0]:
# Save the filtered DataFrame to a CSV file
df_filtered.to_csv('filtered_inaturalist_data.csv', index=False)

print("Filtered data saved to 'filtered_inaturalist_data.csv'.")